Data Augmentation

In [6]:
import numpy as np
import cv2
import random
import threading
import os
import time
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def Rotation(image):
    rows,cols,channel = image.shape
    angle = np.random.uniform(low=-20.0, high=20.0)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    dst = cv2.warpAffine(image, M, (cols,rows))
    return dst

def Translate(image):
    rows,cols,channel = image.shape
    x_ = rows*0.1
    y_ = cols*0.1
    xscale = np.random.uniform(0.6, 1.2)
    yscale = np.random.uniform(0.6, 1.2)
    x = np.random.randint(-x_, x_)
    y = np.random.randint(-y_, y_)
    M = np.float32([[xscale, 0, x], [0, xscale, y]])
    dst = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return dst


def Crop(image):
    rows,cols,channel = image.shape
    L_delta = int(np.random.uniform(1, cols*0.15))
    R_delta = int(np.random.uniform(1, cols*0.15))
    U_delta = int(np.random.uniform(1, rows*0.15))
    D_delta = int(np.random.uniform(1, rows*0.15))
    TOP = 0 + L_delta
    DOWN = rows - R_delta
    LEFT = 0 + U_delta
    RIGHT = cols - D_delta
    crop_img = image[TOP:DOWN, LEFT:RIGHT]
    dst = cv2.copyMakeBorder(crop_img, L_delta, R_delta, U_delta , D_delta, cv2.BORDER_CONSTANT, value=(0, 0, 0, 0))
    return dst

def Hsv(image):
    hue_vari = 1
    sat_vari = 0.5
    val_vari = 0.5
    hue_delta = np.random.randint(-hue_vari, hue_vari)
    sat_mult = 1 + np.random.uniform(-sat_vari, sat_vari)
    val_mult = 1 + np.random.uniform(-val_vari, val_vari)

    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float)
    img_hsv[:, :, 0] = (img_hsv[:, :, 0] + hue_delta) % 180
    img_hsv[:, :, 1] *= sat_mult
    img_hsv[:, :, 2] *= val_mult
    img_hsv[img_hsv > 255] = 255
    
    dst = cv2.cvtColor(np.round(img_hsv).astype(np.uint8), cv2.COLOR_HSV2BGR)
    return dst

def Gamma(image):
    gamma_vari = 0.15
    log_gamma_vari = np.log(gamma_vari)
    alpha = np.random.uniform(-log_gamma_vari, log_gamma_vari)
    gamma = np.exp(alpha)
    gamma_table = [np.power(x / 255.0, gamma) * 255.0 for x in range(256)]
    gamma_table = np.round(np.array(gamma_table)).astype(np.uint8)
    dst = cv2.LUT(image, gamma_table)
    return dst


def Motion_blur(image):
    image = np.array(image)
    degree_ = 25
    angle_ = 45
    degree = int(np.random.uniform(1, degree_))
    angle = int(np.random.uniform(-angle_, angle_))
    M = cv2.getRotationMatrix2D((degree / 2, degree / 2), angle, 1)
    motion_blur_kernel = np.diag(np.ones(degree))
    motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, M, (degree, degree))
    motion_blur_kernel = motion_blur_kernel / degree
    blurred = cv2.filter2D(image, -1, motion_blur_kernel)
    cv2.normalize(blurred, blurred, 0, 255, cv2.NORM_MINMAX)
    dst = np.array(blurred, dtype=np.uint8)
    return dst

def Gaussian_blur(image):

    kernel = [random.randint(1, 50) * 2 + 1 for x in range(1)]
    dst = cv2.GaussianBlur(image, ksize=(kernel[0], kernel[0]), sigmaX=0, sigmaY=0)
    return dst


def imageProcessing(image, image_name, function, times, write_path):  
    for _i in range(0, times, 1):
        if (function == 'Rotation'):
            new_image = Rotation(image)
        elif (function == 'Crop'):
            new_image = Crop(image)
        elif (function == 'Hsv'):
            new_image = Hsv(image)
        elif (function == 'Gamma'):
            new_image = Gamma(image)
        elif (function == 'Motion_blur'):
            new_image = Motion_blur(image)
        elif (function == 'Gaussian_blur'):
            new_image = Gaussian_blur(image)
        elif (function == 'Translate'):
            new_image = Translate(image)
        else:
            new_image = image
        
        path = write_path+image_name+"_"+function+"_"+str(_i)+".JPG"
        cv2.imwrite(path ,new_image)


In [8]:
Init_Path = "C:/Users/xwen2/Pictures/100cent/"
Mid_Path = "C:/Users/xwen2/Pictures/Midway/"
Final_Path = "C:/Users/xwen2/Pictures/A_100cent/"

In [9]:

filelist = os.listdir(Init_Path)
print("Total number of pictures(Init_path): ", len(filelist))
for item in filelist:
    Image = cv2.imread(Init_Path + item, 1)
    imageProcessing(Image, item, " "       ,  1, Mid_Path)
    imageProcessing(Image, item, "Gamma"   ,  1, Mid_Path)
    imageProcessing(Image, item, "Hsv"     ,  1, Mid_Path)
print("Done !!!")


Total number of pictures(Init_path):  154
Done !!!


In [10]:

filelist = os.listdir(Mid_Path)
print("Total number of pictures (Midway): ", len(filelist))
for item in filelist:
    Image = cv2.imread(Mid_Path + item, 1)
    imageProcessing(Image, item, "Motion_blur"  , 1, Mid_Path)  
print("Done !!!")


Total number of pictures (Midway1):  462
Done !!!


In [14]:

filelist = os.listdir(Mid_Path)
print("Total number of pictures (Midway): ", len(filelist))
for item in filelist:
    Image = cv2.imread(Mid_Path + item,1)
    imageProcessing(Image, item, " "       ,  1, Final_Path)
    imageProcessing(Image, item, "Rotation",  1, Final_Path)
print("Done !!!")
    

Total number of pictures (Midway1):  924
Done !!!


In [15]:

filelist = os.listdir(Final_Path)
print("Total number of pictures (Final_Path): ", len(filelist))
for item in filelist:
    Image = cv2.imread(Final_Path + item, 1) 
    imageProcessing(Image, item, " ", 1,Final_Path)
    imageProcessing(Image, item, "Translate", 2,Final_Path)
print("Augmentation process done !!!")


Total number of pictures (Midway2):  2772


KeyboardInterrupt: 

In [ ]:
train_dir = 'D:/picture/train/'
#存放用来训练的图片的路径

def get_files(file_dir):
    A5 = []
    label_A5 = []
    A6 = []
    label_A6 = []
    SEG = []
    label_SEG = []
    SUM = []
    label_SUM = []
    LTAX1 = []
    label_LTAX1 = []
    #定义存放各类别数据和对应标签的列表，列表名对应你所需要分类的列别名
    #A5，A6等是我的数据集中要分类图片的名字


    for file in os.listdir(file_dir):
        name = file.split(sep='.')
        if name[0]=='A5':
            A5.append(file_dir+file)
            label_A5.append(0)
        elif name[0] == 'A6':
            A6.append(file_dir+file)
            label_A6.append(1)
        elif name[0]=='LTAX1':
            LTAX1.append(file_dir+file)
            label_LTAX1.append(2)
        elif name[0] == 'SEG':
            SEG.append(file_dir+file)
            label_SEG.append(3)
        else:
            SUM.append(file_dir+file)
            label_SUM.append(4)
       #根据图片的名称，对图片进行提取，这里用.来进行划分
       ###这里一定要注意，如果是多分类问题的话，一定要将分类的标签从0开始。这里是五类，标签为0，1，2，3，4。我之前以为这个标签应该是随便设置的，结果就出现了Target[0] out of range的错误。

    print('There are %d A5\nThere are %d A6\nThere are %d LTAX1\nThere are %d SEG\nThere are %d SUM' \
          %(len(A5),len(A6),len(LTAX1),len(SEG),len(SUM)))
   #打印出提取图片的情况，检测是否正确提取

    image_list = np.hstack((A5,A6,LTAX1,SEG,SUM))
    label_list = np.hstack((label_A5,label_A6,label_LTAX1,label_SEG,label_SUM))
    #用来水平合并数组

    temp = np.array([image_list,label_list])
    temp = temp.transpose()
    np.random.shuffle(temp)

    image_list = list(temp[:,0])
    label_list = list(temp[:,1])
    label_list = [int(i) for i in label_list]

    return  image_list,label_list
